In [ ]:
def Untargeted_Attack_JSMA(img,model,EPOCH,PerRge):
    orig_label=np.argmax(model(img).data.cpu().numpy())
    print("orig_label={}:{}".format(orig_label,class_label[orig_label]))
    
    origlabel=Variable(torch.Tensor([float(orig_label)]).to(device).long())
    loss_func = torch.nn.CrossEntropyLoss()
    mask = np.ones_like(img.data.cpu().numpy())

    theta=0.05
    max_=3.0
    min_=-3.0
    imgo=img.data.cpu().numpy().copy()
    img_max= imgo+PerRge
    img_min=imgo-PerRge    

    img.requires_grad = True
    for param in model.parameters():
        param.requires_grad = False
        
    for epoch in range(EPOCH):
        output = model(img.to(device))
        label=np.argmax(output.data.cpu().numpy())           
        loss = loss_func(output, origlabel)
        if label != orig_label:
            print(f"Epoch {epoch}")
            print("label:{}->{} loss={:.6f}\n-------------------------------".format(orig_label,label,loss)) 
            print("Attack successfully!")
            break
        
        
        if epoch%10==0:
            print(f"Epoch {epoch}")
            print("label:{}->{} loss={:.6f}\n-------------------------------".format(orig_label,label,loss))
            
        zero_gradients(img)
        idx, pix_sign=max_grad(output, img,orig_label, mask)
        
        img.data[idx]=img.data[idx]-pix_sign*theta#* (max_ - min_)                 
        
       
        if (img.data[idx]<=min_) or (img.data[idx]>=max_):
            print("idx_1={} over {}".format(idx,img.data[idx]))
            mask[idx]=0
            img.data[idx]=np.clip(img.data[idx].cpu(), min_, max_)
        
        if (img.data[idx]<=img_min[idx]) or (img.data[idx]>=img_max[idx]):
            print("idx_2={} over {}".format(idx,img.data[idx]))
            mask[idx]=0
            img.data[idx]=np.clip(img.data[idx].cpu(), img_min[idx], img_max[idx])

    preds_=Func.softmax(model(img),dim=1)
    probs_,idx_=preds_.data.squeeze().sort(0,True)
    probs_=probs_.data.cpu().numpy()
    idx_=idx_.data.cpu().numpy()
    for k_ in range(10):
        print('{}, {},  {:.5f}, {}'.format(k_+1, idx_[k_], probs_[k_], class_label[idx_[k_]]))
          
    adv= img.data.cpu().numpy()[0]
    adv = adv.transpose(1, 2, 0)
    adv = (adv * std) + mean
    adv = adv * 256.0
    adv = np.clip(adv, 0, 255).astype(np.uint8)    
    
        
    print("original {}:{}".format(orig_label,class_label[orig_label]))
    print("current {}:{}".format(label,class_label[label]))          
    return orig_label,adv,label,epoch+1